In [1]:
import cohere

In [2]:
client = cohere.Client()

In [5]:
response = client.chat(
    message="is it too late to join the course?"
)

In [7]:
response.text

"I don't have enough information to answer your question. It depends on the specific course and its enrollment policies. Some courses may have strict deadlines for registration, while others may allow late additions. It's best to check with the course organizer or institution directly to find out if you can still join. They can provide you with the most accurate and up-to-date information regarding enrollment deadlines and any possible exceptions."

## RAG Intro

In [8]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-06-28 20:10:43--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-06-28 20:10:44 (27.2 MB/s) - ‘minsearch.py’ saved [3832/3832]



In [9]:
import minsearch

In [10]:
!wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/documents.json

--2024-06-28 20:13:33--  https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/documents.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 658332 (643K) [text/plain]
Saving to: ‘documents.json’

documents.json      100%[===================>] 642.90K  --.-KB/s    in 0.03s   

2024-06-28 20:13:33 (18.2 MB/s) - ‘documents.json’ saved [658332/658332]



In [11]:
import json

In [21]:
with open('01-intro/documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [23]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [24]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [25]:
index = minsearch.Index(
    text_fields=['question','text','section'],
    keyword_fields=['course']
)

In [26]:
q = 'the course has already started, can I still enroll?'

In [27]:
index.fit(documents)

In [30]:
boost = {'question': 3.0, 'section': 0.5}

results=index.search(
    query=q,
    filter_dict={'course': 'data-engineering-zoomcamp'},
    boost_dict=boost,
    num_results=5
)

In [31]:
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

## Generating Answers with Cohere

In [32]:
response = client.chat(
    message=q
)

In [34]:
response.text

'Yes, you can still enroll in the course even if it has already started. However, please note that you may have missed some of the course content and will need to catch up on your own time. It is also a good idea to reach out to the instructor to see if they have any recommendations for catching up with the course material.'

In [42]:
prompt_template="""
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only facts from the CONTEXT when answering the QUESTION.
If the CONTEXT doesn't contain the answer, output NONE

QUESTION:{question}

CONTEXT: {context}
""".strip()

In [39]:
context = ""

for doc in results:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

In [40]:
print(context)

section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.

section: General course-related questions
question: Course - When will the course start?
answer: The purpose of this document is to capture frequently asked technical questions
The exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start wit

In [45]:
prompt=prompt_template.format(question=q, context=context).strip()

In [46]:
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only facts from the CONTEXT when answering the QUESTION.
If the CONTEXT doesn't contain the answer, output NONE

QUESTION:the course has already started, can I still enroll?

CONTEXT: section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final ca

In [48]:
response=client.chat(
    message=prompt
)
response.text

'Yes, you can still enroll and submit homework. However, be mindful of the deadlines for turning in final projects.'

## Modularization

In [49]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results=index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )
    return results

In [62]:
def build_prompt(query, search_results):
    prompt_template="""
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only facts from the CONTEXT when answering the QUESTION.
    If the CONTEXT doesn't contain the answer, output NONE
    
    QUESTION:{question}
    
    CONTEXT: {context}
    """.strip()

    context = ""

    for doc in results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

    prompt=prompt_template.format(question=query, context=search_results).strip()
    return prompt

In [63]:
def llm(prompt):
    response=client.chat(
    message=prompt
    )
    return response.text

In [64]:
query='how do i run kafka?'
search_results = search(query)
prompt=build_prompt(query, search_results)
answer=llm(prompt)
answer

"To run Kafka in the terminal, you can follow these steps, depending on the specific context: \n\n**For Java Kafka:**\nIn the project directory, run the following command:\n```\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n```\n\n**For Python Kafka:**\n1. Create a virtual environment and install the necessary packages:\n```\npython -m venv env\nsource env/bin/activate\npip install -r ../requirements.txt\n```\n2. Activate the virtual environment each time before running the Python files:\n```\nsource env/bin/activate\n```\n3. Deactivate the virtual environment when you're done:\n```\ndeactivate\n```\nNote: The path to activate the virtual environment might be slightly different on Windows (env/Scripts/activate).\n\n**For Python Kafka Permission Denied Error:**\nRun the following command in the terminal, in the /docker/spark directory:\n```\nchmod +x build.sh\n```\n\n**For ModuleNotFoundError regarding kafka.vendor.six.moves:**\nUse the

In [65]:
print(answer)

To run Kafka in the terminal, you can follow these steps, depending on the specific context: 

**For Java Kafka:**
In the project directory, run the following command:
```
java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java
```

**For Python Kafka:**
1. Create a virtual environment and install the necessary packages:
```
python -m venv env
source env/bin/activate
pip install -r ../requirements.txt
```
2. Activate the virtual environment each time before running the Python files:
```
source env/bin/activate
```
3. Deactivate the virtual environment when you're done:
```
deactivate
```
Note: The path to activate the virtual environment might be slightly different on Windows (env/Scripts/activate).

**For Python Kafka Permission Denied Error:**
Run the following command in the terminal, in the /docker/spark directory:
```
chmod +x build.sh
```

**For ModuleNotFoundError regarding kafka.vendor.six.moves:**
Use the following command to install kaf

In [66]:
def rag(query):
    search_results = search(query)
    prompt=build_prompt(query, search_results)
    answer=llm(prompt)
    return answer

In [67]:
print(rag(query))

To run Kafka in the terminal, you can follow these steps, depending on the specific context: 

**For Java Kafka:**
In the project directory, run the following command:
```
java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java
```

**For Python Kafka:**
1. Create a virtual environment and install the necessary packages:
```
python -m venv env
source env/bin/activate
pip install -r ../requirements.txt
```
2. Activate the virtual environment each time before running Python files:
```
source env/bin/activate
```
3. Deactivate the virtual environment when you're done:
```
deactivate
```
Note: The path for activation might vary slightly on Windows (env/Scripts/activate).

**Additional Notes:**
- Ensure that Docker images are up and running before creating the virtual environment.
- To fix the "ModuleNotFoundError: No module named 'kafka.vendor.six.moves'" error, use the following command:
```
pip install kafka-python-ng
```


In [68]:
print(rag('the course has already started, can i still enroll?'))

Yes, you can still enroll in the course even after it has started. You are still eligible to submit homework, but be mindful of the deadlines for turning in final projects.


## Elasticsearch

In [69]:
from elasticsearch import Elasticsearch

In [70]:
es = Elasticsearch('http://localhost:9200')

In [73]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name='course-questions'

es.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [74]:
from tqdm.auto import tqdm

/usr/local/python/3.10.13/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [75]:
for doc in tqdm(documents):
    es.index(index=index_name, document=doc)

100%|█████████████████████████████████████████████████████| 948/948 [00:23<00:00, 41.12it/s]


In [76]:
query = 'I just discovered the course. Can I still join it?'

In [77]:
search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^3", "text", "section"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "data-engineering-zoomcamp"
                }
            }
        }
    }
}

In [79]:
response=es.search(index=index_name, body=search_query)

In [82]:
result_docs = []
for hit in response['hits']['hits']:
    result_docs.append(hit['_source'])

In [83]:
result_docs

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (insta

In [84]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response=es.search(index=index_name, body=search_query)

    result_docs = []
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    return result_docs

In [85]:
def rag2(query):
    search_results = elastic_search(query)
    prompt=build_prompt(query, search_results)
    answer=llm(prompt)
    return answer

In [86]:
rag2(query)

"Yes, you can still join the course even if you haven't registered. You can submit homework and follow the course at your own pace, even after it finishes. However, keep in mind the deadlines for turning in final projects."

# Homework

## Q1

In [89]:
es.info()

ObjectApiResponse({'name': '33154be37275', 'cluster_name': 'docker-cluster', 'cluster_uuid': '3sXDSWhuQiuEEcexqYiXMA', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

## Q2

Adding data to elasticsearch requires `index`

## Q3

In [90]:
query = 'How do I execute a command in a running docker container?'

In [100]:
search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
            #"filter": {
            #    "term": {
            #        "course": "data-engineering-zoomcamp"
            #    }
            #}
        }
    }
}

response=es.search(index=index_name, body=search_query)

In [101]:
dict(response)

{'took': 7,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 865, 'relation': 'eq'},
  'max_score': 84.050095,
  'hits': [{'_index': 'course-questions',
    '_id': 'Lz-8YJABMkm3_fCZnBYP',
    '_score': 84.050095,
    '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
     'section': '5. Deploying Machine Learning Models',
     'question': 'How do I debug a docker container?',
     'course': 'machine-learning-zoomcamp'}},
   {'_index': 'course-questions',
    '_id': 'PT-8YJABMkm3_fCZbBSs',
    '_score': 75.54128,
    '_source': {'text': 'In case running pgcli  locally causes issues or you do not want to install it locally you c

## Q4

In [102]:
search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}

response=es.search(index=index_name, body=search_query)

In [103]:
dict(response)

{'took': 7,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 345, 'relation': 'eq'},
  'max_score': 84.050095,
  'hits': [{'_index': 'course-questions',
    '_id': 'Lz-8YJABMkm3_fCZnBYP',
    '_score': 84.050095,
    '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
     'section': '5. Deploying Machine Learning Models',
     'question': 'How do I debug a docker container?',
     'course': 'machine-learning-zoomcamp'}},
   {'_index': 'course-questions',
    '_id': 'Tj-8YJABMkm3_fCZnhb6',
    '_score': 51.04628,
    '_source': {'text': "You can copy files from your local machine into a Docker container using the docker cp comm

## Q5

In [107]:
results_list = dict(response)['hits']['hits']
results_list

[{'_index': 'course-questions',
  '_id': 'Lz-8YJABMkm3_fCZnBYP',
  '_score': 84.050095,
  '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
   'section': '5. Deploying Machine Learning Models',
   'question': 'How do I debug a docker container?',
   'course': 'machine-learning-zoomcamp'}},
 {'_index': 'course-questions',
  '_id': 'Tj-8YJABMkm3_fCZnhb6',
  '_score': 51.04628,
  '_source': {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_

In [108]:
context_template = """
Q: {question}
A: {text}
""".strip()

In [117]:
def build_context(results_list):
    context_template = """
    Q: {question}
A: {text}
""".strip()
    context = ""
    for result in results_list:
        question = result['_source']['question']
        answer = result['_source']['text']
        context += context_template.format(question=question, text=answer) + "\n\n"

    return context.strip()

In [118]:
context = build_context(results_list)
print(context)

Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:/path/in/container
Hrithik Kumar Advani

Q: How do I copy files from a different folder into docker container’s working directory?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to 

In [119]:
len(context)

2470

In [120]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [121]:
prompt=prompt_template.format(question=query, context=context).strip()

In [122]:
len(prompt)

2714

## Q6

In [124]:
tokenize = client.tokenize(text=prompt, model='command')

In [126]:
print(tokenize)

tokens=[3375, 2516, 1675, 3428, 6955, 9940, 21, 33183, 1682, 75089, 4222, 3103, 1748, 1682, 7130, 21477, 1855, 1682, 26446, 7671, 21, 206, 10431, 2222, 1682, 11285, 1855, 1682, 7130, 21477, 2112, 22968, 1682, 75089, 4222, 21, 206, 206, 7047, 9759, 4222, 33, 2852, 1978, 1744, 13455, 1675, 5220, 1703, 1675, 4494, 28769, 9199, 38, 206, 206, 46451, 33, 206, 56, 33, 2852, 1978, 1744, 16282, 1675, 28769, 9199, 38, 206, 40, 33, 28014, 1682, 9199, 4406, 1703, 11947, 6735, 1708, 58070, 1682, 6271, 4827, 19, 2006, 1769, 1784, 7463, 1675, 38366, 5220, 21, 206, 18963, 2634, 2050, 1689, 2483, 9597, 4827, 38366, 2211, 6162, 37, 206, 2915, 1682, 9199, 1740, 3599, 4494, 19, 13455, 1675, 5220, 1703, 1682, 3242, 9199, 33, 206, 18963, 19646, 1773, 10546, 1682, 9199, 20, 1736, 16, 206, 18963, 7701, 2050, 1689, 2211, 11942, 20, 1736, 37, 38366, 206, 15, 32592, 1839, 1760, 35386, 16, 206, 206, 56, 33, 2852, 1978, 1744, 4498, 5426, 1855, 2131, 2971, 4412, 1705, 28769, 9199, 38, 206, 40, 33, 2306, 1863, 4498,

In [128]:
dict(tokenize)

{'tokens': [3375,
  2516,
  1675,
  3428,
  6955,
  9940,
  21,
  33183,
  1682,
  75089,
  4222,
  3103,
  1748,
  1682,
  7130,
  21477,
  1855,
  1682,
  26446,
  7671,
  21,
  206,
  10431,
  2222,
  1682,
  11285,
  1855,
  1682,
  7130,
  21477,
  2112,
  22968,
  1682,
  75089,
  4222,
  21,
  206,
  206,
  7047,
  9759,
  4222,
  33,
  2852,
  1978,
  1744,
  13455,
  1675,
  5220,
  1703,
  1675,
  4494,
  28769,
  9199,
  38,
  206,
  206,
  46451,
  33,
  206,
  56,
  33,
  2852,
  1978,
  1744,
  16282,
  1675,
  28769,
  9199,
  38,
  206,
  40,
  33,
  28014,
  1682,
  9199,
  4406,
  1703,
  11947,
  6735,
  1708,
  58070,
  1682,
  6271,
  4827,
  19,
  2006,
  1769,
  1784,
  7463,
  1675,
  38366,
  5220,
  21,
  206,
  18963,
  2634,
  2050,
  1689,
  2483,
  9597,
  4827,
  38366,
  2211,
  6162,
  37,
  206,
  2915,
  1682,
  9199,
  1740,
  3599,
  4494,
  19,
  13455,
  1675,
  5220,
  1703,
  1682,
  3242,
  9199,
  33,
  206,
  18963,
  19646,
  1773,
  10546,
